<a href="https://colab.research.google.com/github/AbuMulla-Mohammad/CVSegmentation/blob/main/Document_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cohere
!pip install instructor
!pip install docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.6/327.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 28.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfull

In [ ]:
!pip install instructor
!pip uninstall generativeai
!pip install generativeai


In [ ]:
!pip install python-docx



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
from pydantic import BaseModel
import cohere
import instructor
from docx import Document
import os
from transformers import pipeline
from pydantic import BaseModel, Field
from typing import List

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
#document_processing
def doc_with_lines(document: str):
    document_lines = document.split("\n")
    document_with_line_numbers = ""
    line2text = {}
    for i, line in enumerate(document_lines):
        document_with_line_numbers += f"[{i}] {line}\n"
        line2text[i] = line
    return document_with_line_numbers, line2text


In [ ]:
#segments_categorizatio

# Load a pre-trained text classification model
classifier = pipeline('text-classification', model='distilbert-base-uncased')
def classify_segment(text: str) -> str:
    result = classifier(text)
    label = result[0]['label']
    return label

def categorize_segments(segments: list) -> list:
    categorized_segments = []
    for segment in segments:
        category = classify_segment(segment['content'])
        # Add category to segment
        segment['category'] = category
        categorized_segments.append(segment)
    return categorized_segments


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
class Section(BaseModel):
    title: str = Field(description="Main topic of this section of the document")
    start_index: int = Field(description="Line number where the section begins")
    end_index: int = Field(description="Line number where the section ends")
    category:str=Field(description="The Category of this section")
class StructuredDocument(BaseModel):
    """Obtains meaningful sections, each centered around a single concept/topic."""
    sections: List[Section] = Field(description="A list of sections of the document")

In [ ]:
api_key="QRgahiOvBH2HBDFdd9ye1veAkzpliD4t2VD4LxiJ"
if not api_key:
    raise ValueError("API key for Cohere is not set in the .env file.")

# Initialize the cohere client with the API key
client = cohere.Client(api_key)

# Apply the patch to the cohere client
# Enables response_model keyword
client = instructor.from_cohere(client)

system_prompt = """
You are a skilled resume analyst tasked with organizing a CV.
Read the document below and extract a StructuredDocument object from it where each section of the CV is clearly defined, categorized, and labeled with an appropriate category.
Each section should represent a distinct part of the CV, such as Education, Experience, Skills, etc.
Your task is to identify the start, end, and the category of each section using the line numbers provided in square brackets (e.g., [1], [2], [3], etc.), and assign a category label to each section based on its content.
Ensure to analyze and extract information from tables as well, as they may contain crucial details about Education, Experience, Skills, or other sections.
"""



def get_structured_document(document_with_line_numbers: str) -> StructuredDocument:
    response = client.chat.completions.create(
        model="command-r-plus",
        response_model=StructuredDocument,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": document_with_line_numbers,
            },
        ],
    )
    return response

def get_sections_text(structured_doc: StructuredDocument, line2text: dict) -> list:
    segments = []
    print("from getsection text function here is thestructured_doc ",structured_doc.dict)
    for s in structured_doc.sections:
        contents = [line2text.get(line_id, '') for line_id in range(s.start_index, s.end_index)]
        segments.append({
            "title": s.title,
            "content": "\n".join(contents),
            "start": s.start_index,
            "end": s.end_index,
            "category":s.category
        })
    for s in structured_doc.sections:
      print(s.category,"\n")
    return segments

def extract_text_from_docx(docx_path: str) -> str:
    doc = Document(docx_path)
    text = []

    # Extract text from paragraphs
    for para in doc.paragraphs:
        text.append(para.text)

    # Extract text from tables
    for table in doc.tables:
        for row in table.rows:
            row_text = []
            for cell in row.cells:
                row_text.append(cell.text)
            text.append("\t".join(row_text))  # Separate columns with a tab

    return "\n".join(text)

# Prompt for user input for DOCX file
docx_path = '/files/SanadAhmad-CV.docx'

# Extract text from the DOCX file
try:
    document = extract_text_from_docx(docx_path)
    document_with_line_numbers, line2text = doc_with_lines(document)
    structured_doc = get_structured_document(document_with_line_numbers)
    segments = get_sections_text(structured_doc, line2text)

    # Print the results
    print("Structured Document:")
    # Use .dict() and convert to JSON manually if necessary
    print(structured_doc.dict(),"______________++++++++++++++++______________")

    print("\nSegments:")
    for segment in segments:
      print(segment,"\n")
    for segment in segments:
        print(f"Title: {segment['title']}")
        print(f"Start: {segment['start']}")
        print(f"End: {segment['end']}")
        print(f"Category: {segment['category']}")
        print("Content:")
        print(segment['content'])
        print("-" * 40)

except Exception as e:
    print(f"An error occurred: {e}")

from getsection text function here is thestructured_doc  <bound method BaseModel.dict of StructuredDocument(sections=[Section(title='Personal Information', start_index=0, end_index=5, category='Contact and Address'), Section(title='Academic Qualifications', start_index=8, end_index=10, category='Education'), Section(title='Publications', start_index=11, end_index=17, category='Research and Publications'), Section(title='Work Experience', start_index=19, end_index=28, category='Employment History'), Section(title='Key Experiences', start_index=30, end_index=35, category='Skills and Expertise'), Section(title='Education Summary', start_index=36, end_index=40, category='Education'), Section(title='Business and Technical Skills', start_index=41, end_index=47, category='Skills and Expertise')])>
Contact and Address 

Education 

Research and Publications 

Employment History 

Skills and Expertise 

Education 

Skills and Expertise 

Structured Document:
{'sections': [{'title': 'Personal In